In [ ]:
! git clone https://github.com/n-waves/multifit.git
! pip install multifit/
! pip install -r multifit/requirements.txt

In [18]:
from google.colab import files
import io
import pandas as pd
from fastai.text import *
import multifit

In [16]:
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['new_myneral99_tweets.csv']))
data.head()

Saving new_myneral99_tweets.csv to new_myneral99_tweets (3).csv


,id,created_at,text
0,1321277704826691585,2020-10-28 02:28:47,Esta se adelantó a navidad
1,1321277405177274369,2020-10-28 02:27:36,@joskere Im just being myself
2,1321277027614367746,2020-10-28 02:26:06,"@joskere Yo, me parece horrendo por dentro y p..."
3,1321276636441051137,2020-10-28 02:24:32,Dios franchin sabe abrir la boca sin dar vergü...
4,1321274889119191042,2020-10-28 02:17:36,Gente grande llorando en televisión...


In [11]:
#data = (TextList.from_df(data, cols='text')
                #.random_split_by_pct(0.1)
                #.label_for_lm()  
                #.databunch(bs=48))

#data.show_batch()

idx,text
0,"a través de sus storys de ig , necesito más y más xxbos xxmaj vi unos videos xxunk de xxunk xxunk y me dió bronca porque antes hablaba mas normal y no se hacía tanto la boluda xxbos xxmaj xxunk por la facultad , xxunk aburrida hace horas . xxmaj así es como xxunk vivir , como el orto . xxbos xxmaj mi hermana : ese feo ? ? ?"
1,"para xxup now ? ? xxbos @amigxcamila xxmaj porque xxunk a los más xxunk pero estaba lleno de xxunk xxbos xxmaj xxunk porque me xxunk cantar mientras xxunk unos xxunk , que más quiere una chica ? xxbos xxmaj la xxunk de xxmaj xxunk xxunk y xxunk xxunk salió 14 millones , el doble de lo que xxunk unos zapatos que uso xxmaj xxunk xxmaj xxunk … https : /"
2,"como "" una xxunk perfecta "" xxbos @juliancapo98 xxmaj hay muchas así que son muy lindas para xxunk como "" xxunk "" , en la m xxunk encima hay una combinación de xxunk xxunk xxbos xxmaj xxunk ? i do n't know her xxbos el problema de xxunk es que xxunk es xxunk y xxunk xxbos xxunk xxmaj vos xxunk que si xxbos xxmaj tengo un xxunk de xxunk que"
3,"xxunk xxbos xxmaj ya tengo xxunk un emoji , por 20 xxmaj lucas xxunk me xxunk todo el xxunk https : / / t.co / xxunk xxbos xxmaj ayer pensaba xxunk temprano para hacer un trabajo pero en un rato de voluntad xxunk lo hice todo de xxunk … https : / / t.co / xxunk xxbos xxup rt xxunk : https : / / t.co / xxunk xxbos @_roberga"
4,"gracioso xxbos xxmaj me xxunk al xxunk https : / / t.co / xxunk xxbos xxunk @telefe xxmaj perdón por xxunk xxunk estúpido , no me había dado cuenta que estaba xxunk con un viejo xxunk u.u xxbos xxmaj dios xxmaj xxunk era literalmente una xxunk que mujer https : / / t.co / xxunk xxbos xxmaj existe algo más retrasado que los xxunk ? xxbos xxunk @telefe xxunk xxmaj"


In [19]:
exp = multifit.from_pretrained("es_multifit_paper_version")
fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)
data_lm = (TextList.from_df(data, cols='text')
                .random_split_by_pct(0.1)
                .label_for_lm()  
                .databunch(bs=48))
learn = exp.finetune_lm.get_learner(data_lm)  
# learn is a preconfigured fastai learner with a pretrained model loaded
learn.fit_one_cycle(10)
learn.save_encoder("enc")

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:217: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


Training args:  {'drop_mult': 0.3, 'true_wd': False, 'wd': 1e-07, 'pretrained': False, 'clip': 0.12} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15, 'tie_weights': True, 'out_bias': True}
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Setting LM training seed seed to 0
Loading pretrained weights:  [PosixPath('/root/.fastai/models/es_multifit_paper_version/lm_best'), PosixPath('/root/.fastai/models/es_multifit_paper_version/itos')]


epoch,train_loss,valid_loss,accuracy,time
0,10.944069,9.416085,0.005804,01:36
1,10.787601,8.956981,0.006845,01:33
2,10.317929,8.007965,0.029762,01:33
3,9.706690,7.206182,0.057143,01:33
4,9.066031,6.517559,0.063095,01:34
5,8.464766,6.006332,0.066964,01:33
6,7.987203,5.730698,0.075446,01:37
7,7.585806,5.590186,0.080060,01:33
8,7.271845,5.505247,0.080655,01:33
9,7.069219,5.557303,0.081399,01:34


In [27]:
print(learn.predict('la puta madre', n_words=20))

la puta madre creen d pero muchas baby mirando buen un habría si es la … entiendo bowie después malo ir me de


In [26]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(1792, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(1792, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): QRNN(
        (layers): ModuleList(
          (0): QRNNLayer(
            (linear): WeightDropout(
              (module): Linear(in_features=800, out_features=4650, bias=True)
            )
          )
        )
      )
      (1): QRNN(
        (layers): ModuleList(
          (0): QRNNLayer(
            (linear): WeightDropout(
              (module): Linear(in_features=1550, out_features=4650, bias=True)
            )
          )
        )
      )
      (2): QRNN(
        (layers): ModuleList(
          (0): QRNNLayer(
            (linear): WeightDropout(
              (module): Linear(in_features=1550, out_features=4650, bias=True)
            )
          )
        )
      )
      (3): QRNN(
        (layers): ModuleList(
          (0): QRNNLayer(
            (linear): WeightDro

In [28]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.519549,4.392770,0.274405,02:16
1,4.958775,3.993084,0.322917,02:16
2,4.639804,3.820355,0.327976,02:13
3,4.389749,3.624072,0.346429,02:14
4,4.176977,3.516666,0.362946,02:13
5,3.992261,3.451342,0.373958,02:15
6,3.819606,3.416001,0.378869,02:16
7,3.668404,3.394577,0.380208,02:13
8,3.537497,3.405631,0.380060,02:13
9,3.432948,3.406989,0.379018,02:13


/usr/local/lib/python3.6/dist-packages/torch/nn/utils/clip_grad.py:24: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  parameters = list(filter(lambda p: p.grad is not None, parameters))
/usr/local/lib/python3.6/dist-packages/torch/nn/utils/clip_grad.py:24: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorc

In [31]:
print(learn.predict('lo que más odio en el mundo es', n_words=20))

lo que más odio en el mundo es alguna para un ser … https : / / t.co / / t.co / MUCHO por el ESO
